In [1]:
from utils import *

## Decompress CommandedScanLaw_001.csv.gz file

In [ ]:
import gzip
import shutil

# Define the input and output file paths
input_gz_file = 'data/CommandedScanLaw_001.csv.gz'
output_csv_file = 'data/CommandedScanLaw_001.csv'

# Open the gzipped file and write its content to the new CSV file
with gzip.open(input_gz_file, 'rb') as f_in:
    with open(output_csv_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

print(f'File {input_gz_file} has been decompressed to {output_csv_file}')

### Load ScanLaw file

In [ ]:
# /Gaia/gedr3/auxiliary/commanded_scan_law/ @ https://cdn.gea.esac.esa.int/Gaia/gedr3/auxiliary/commanded_scan_law/
local_csv_filename = "data/CommandedScanLaw_001.csv"

# Load the scanning law times from a local CSV file with the specified filename and version
dr3_sl = scanninglaw.times.Times(map_fname=local_csv_filename, version='dr3_nominal')


In [ ]:
dr3Period=34/12 # Calculate the period in years by dividing 34 by 12
mas=astromet.mas # conversion from degrees to milli-arcseconds

## Generate nTest simulated star systems

##### Simulation of 

##### REFERENCE: code from astromet - https://github.com/zpenoyre/astromet.py/blob/master/gaiaBinarySynth.ipynb

In [ ]:
nTest=10000
dataNames=('RA','Dec','pmRA','pmDec','pllx',
           'M_tot','q','l','a','e','P','tPeri',
           'vTheta','vPhi','vOmega',
           'predict_dTheta','simple_dTheta',
           'N_obs','sigma_al','sigma_ac',
           'fit_ra','fit_dec','fit_pmrac','fit_pmdec','fit_pllx',
           'sigma_rac','sigma_dec','sigma_pmrac','sigma_pmdec','sigma_pllx',
           'N_vis','frac_good','AEN','UWE'
          )
          
allData=astropy.table.Table(names=dataNames)

alError=1
#acError=0.3  -- currently ignoring across scan measurements (~negligible change)

# we'll generate nTest sets of parameters, and test each with a 0, 2 ,5 and 10 mSun BH
for i in tqdm(range(nTest)):
    allData.add_row()
    thisRow=allData[i]
    
    params=astromet.params()
    params.ra=360*np.random.rand()
    params.dec=np.arcsin(-1+2*np.random.rand())*180/np.pi
    
    c=Source(params.ra,params.dec,unit='deg')
    sl=dr3_sl(c, return_times=True, return_angles=True)
    ts=2010+np.squeeze(np.hstack(sl['times']))/365.25
    sort=np.argsort(ts)
    ts=np.double(ts[sort])
    
    phis=np.squeeze(np.hstack(sl['angles']))[sort]
    
    params.parallax=10*np.power(np.random.rand(),-1/3) # all within 100 pc
    params.pmrac=params.parallax*(1)*np.random.randn()
    params.pmdec=params.parallax*(1)*np.random.randn()
    params.period=10**(-1.5+3*np.random.rand()) # periods between 0.03 and 30 years
    params.l=np.random.rand() # uniform light ratio
    params.q=4*np.random.rand()**2 # mass ratios between 0 and 4 (half less than 1)
    params.a=10*np.random.rand()**2
    params.e=np.random.rand()
    params.vtheta=np.arccos(-1+2*np.random.rand())
    params.vphi=2*np.pi*np.random.rand()
    params.vomega=2*np.pi*np.random.rand()
    orbitalPhase=np.random.rand() # fraction of an orbit completed at t=0
    params.tperi=params.period*orbitalPhase
    
    thisRow['RA']=1.*params.ra
    thisRow['Dec']=1.*params.dec
    thisRow['pmRA']=params.pmrac
    thisRow['pmDec']=params.pmdec
    thisRow['pllx']=params.parallax
    thisRow['M_tot']=4*(np.pi**2)*astromet.Galt/((params.period**2)*(params.a**3))
    thisRow['q']=params.q
    thisRow['l']=params.l
    thisRow['a']=params.a
    thisRow['e']=params.e
    thisRow['P']=params.period
    thisRow['tPeri']=1.*params.tperi
    thisRow['vTheta']=params.vtheta
    thisRow['vPhi']=params.vphi
    thisRow['vOmega']=params.vomega
    thisRow['sigma_al']=alError
    #thisRow['sigma_ac']=acError

    trueRacs,trueDecs=astromet.track(ts,params)

    # added .astype(float) to avoid astromet error
    phis = phis.astype(float)
    
    t_obs,x_obs,phi_obs,rac_obs,dec_obs=astromet.mock_obs(ts,phis,trueRacs,trueDecs,err=alError)
    
    fitresults=astromet.fit(t_obs,x_obs,phi_obs,alError,params.ra,params.dec)
    results=astromet.gaia_results(fitresults)
    
    # print('ra, dec, pllx, pmrac, pmdec ',params.ra,params.dec,params.parallax,params.pmrac,params.pmdec)
    # print(results)
    
    # bug somewhere in these
    #thisRow['simple_dTheta']=astromet.dtheta_simple(params)
    #thisRow['predict_dTheta']=astromet.dtheta_full(params,np.min(ts),np.max(ts))  
    
    thisRow['fit_ra']=results['ra']
    thisRow['fit_dec']=results['dec']
    thisRow['fit_pmrac']=results['pmra']
    thisRow['fit_pmdec']=results['pmdec']
    thisRow['fit_pllx']=results['parallax']

    thisRow['sigma_rac']=results['ra_error']
    thisRow['sigma_dec']=results['dec_error']
    thisRow['sigma_pmrac']=results['pmra_error']
    thisRow['sigma_pmdec']=results['pmdec_error']
    thisRow['sigma_pllx']=results['parallax_error']

    # results['UWE'] --> results['uwe']
    thisRow['UWE']=results['uwe']

    thisRow['N_obs']=results['astrometric_n_obs_al']
    #thisRow['frac_good']=results['astrometric_n_good_obs_al']/results['astrometric_n_obs_al']
    thisRow['N_vis']=results['visibility_periods_used']
    #thisRow['AEN']=results['astrometric_excess_noise']

In [ ]:
# Convert the astropy table to a pandas DataFrame
df = allData.to_pandas()

# Export the DataFrame to a CSV file named '100k.csv' in the current folder
df.to_csv('data/100k.csv', index=False)

# print the column names
print(df.columns)


### Import GAIA DR3 data

In [ ]:
from astroquery.gaia import Gaia

# Login into ESA Gaia Data Archive (create login if you have not) to run asynchronous queries using your user quota
# credentials file should contain two lines: username and password

# Read more about astroquery and Gaia here:
# https://astroquery.readthedocs.io/en/latest/gaia/gaia.html

# Gaia.login(credentials_file='/Users/vasily/work/python/gaia_archive_credentials')

In [ ]:
# Query Gaia DR3 to select N random sources with given conditions
# ---------------------------------------------------------------
# For n = 1e+6 sources, the query might take ~10-15 min

# n = number of random sources
# mag_lim = limiting magnitude in G
# blim = limiting |b|
# par_sn = threshold in parallax S/N

n = 1000000

# Query
# Make sure that commands inside the query do not stick together, print to check
q = f"""
SELECT TOP {n} source_id, ra, dec, phot_g_mean_mag, bp_rp, parallax, pmra, pmdec, ruwe, 
               radial_velocity_error, rv_method_used, rv_nb_transits, 
               non_single_star, radial_velocity, parallax_error 
FROM gaiadr3.gaia_source 
WHERE radial_velocity_error > 0 
  AND rv_nb_transits > 3 
  AND ruwe > 0 
  AND parallax / parallax_error > 10 
  AND rv_method_used = 1 
  AND phot_g_mean_mag IS NOT NULL
  AND bp_rp IS NOT NULL 
ORDER BY source_id
"""

# Run query
job = Gaia.launch_job_async(q)
r = job.get_results()

# Save the query results to a local CSV file
r.write('data/gaia_dr3.csv', overwrite=True)

### nss_two_body_orbit

In [ ]:
# Query Gaia DR3 to select N random sources with given conditions
# ---------------------------------------------------------------
# For n = 1e+6 sources, the query might take ~10-15 min

# n = number of random sources
# mag_lim = limiting magnitude in G
# blim = limiting |b|
# par_sn = threshold in parallax S/N

n = 10000000

# Query
# Make sure that commands inside the query do not stick together, print to check
q = f"""
SELECT TOP {n} dec, period, arg_periastron, source_id, pmdec, center_of_mass_velocity, \
               nss_solution_type, t_periastron, mass_ratio,\
               ra, semi_amplitude_primary, inclination, pmra, eccentricity \
FROM gaiadr3.nss_two_body_orbit 
WHERE nss_solution_type = 'AstroSpectroSB1' \
OR nss_solution_type = 'Orbital' \
ORDER BY source_id
"""

# Run query
job = Gaia.launch_job_async(q)
r = job.get_results()

# Save the query results to a local CSV file
r.write('data/nss_two_body_orbit.csv', overwrite=True)